<a href="https://colab.research.google.com/github/Barryugo/JRZY/blob/main/Chat1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.0 MB/s eta 0:00:00


In [2]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Load your sports dataset
from google.colab import files
uploaded = files.upload()

Saving Question_Answer_Dataset_v1.2.tar.gz to Question_Answer_Dataset_v1.2.tar.gz


In [7]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer)) # Resize the model's embeddings

dataset = pd.read_csv('scraped_data.csv')

# Preparing the inputs for training
max_length = 512 
inputs = tokenizer(dataset["text"].tolist(), max_length=max_length, return_tensors="pt", padding=True, truncation=True)
labels = inputs.input_ids

# Fine-tuning the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# mixed precision training
scaler = torch.cuda.amp.GradScaler()

# gradient accumulation
batch_size = 8 
num_batches = len(inputs) // batch_size
accumulation_steps = 8 
for epoch in range(10):
  model.train()
  for i in range(num_batches):
    input_ids = inputs.input_ids[i*batch_size:(i+1)*batch_size].to('cuda')
    labels_batch = labels[i*batch_size:(i+1)*batch_size].to('cuda')

    # Clear gradients
    model.zero_grad()

    # Forward pass and loss calculation
    with torch.cuda.amp.autocast():
        outputs = model(input_ids=input_ids, labels=labels_batch)
        loss = outputs.loss / accumulation_steps

    # Backward pass
    scaler.scale(loss).backward()

    # Gradient accumulation
    if (i+1) % accumulation_steps == 0 or i == num_batches-1:
        scaler.step(optimizer)
        scaler.update()

    # Log the loss
    if (i+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Batch {i+1}/{num_batches}, Loss: {loss.item()}")

  # Save the model after each epoch
  torch.save(model.state_dict(), f"gpt2_{epoch+1}.pt")


Using pad_token, but it is not set yet.


In [15]:
prompt = "Hello, how are you today?"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1024, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you today?
 37
 The The 35 35 35 7 37
